# Data Anaysis Project part1:

I am using Vancouver rental ads on craigslist to analyze rental market. Later, I will create the class to be able to analyze dynamically. Also, I will start collecting the data and save them to the CSV, so I can analyze the trend of the rental market as well. 

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so I can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)


#### Review Google Data Analysis course and create checklist:
* Ask effective questions
* Define the problems
* Using structured thinking
* Communicate with others



## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>


#### Review from Google Data Analysis course:
* Understand how data is generated and collect. 
    * The data is collected from Craigslist ads. 
* Identify and use different data formats, types, and structures. 
* Make sure data is unbiased and credible. 
    * The data is live advertisement, so it is great to see today's market price. However, craigslist doesn't seems today's most favorite site to advertize so it might not accurately show the reality. 
* Organize and protect data. 

In [1]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [2]:
class Url:
    def __init__(self, city_name, sub_city=None, num_post=0):
        self.city_name = city_name
        self.sub_city = sub_city
        self.num_post = num_post
        self.url = self.get_url()
        
    def get_url(self):
        if self.city_name == 'Abbotsford':
            url = f'https://{self.city_name.lower()}.craigslist.org/search/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=&max_price=&availabilityMode=0&sale_date=all+dates'
        else:
            url = f'https://{self.city_name.lower()}.craigslist.org/search/{self.sub_city}/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=&max_price=&availabilityMode=0&sale_date=all+dates'
        return url 

In [3]:
# Vanvouver rental housing
url = Url("Vancouver", 'van')
url_ = url.url

response = get(url_)


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
121


In [4]:
%%time
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
sub_city = 'van'
response = get(url_)
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []
post_links = []

for page in pages:
    
    # get request
    url = Url(city_name=city, sub_city=sub_city, num_post=page)
    url_ = url.url
    response = get(url_)
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            # Post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        #print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
CPU times: total: 4.11 s
Wall time: 47.5 s


In [5]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'City Code': city.upper()[:3],
                   'Area Code': sub_city.upper(),
                   'Post Title': post_titles,
                   'Post URL': post_links,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3050 entries, 0 to 3049
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3050 non-null   object 
 1   City Code      3050 non-null   object 
 2   Area Code      3050 non-null   object 
 3   Post Title     3050 non-null   object 
 4   Post URL       3050 non-null   object 
 5   Neighborhood   3050 non-null   object 
 6   Bedroom        2830 non-null   object 
 7   SQFT           2693 non-null   float64
 8   Price          3050 non-null   int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 214.6+ KB
None


,Post Datetime,City Code,Area Code,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-15 12:41,VAN,VAN,"2 elevators, 24 hour on-site professional mana...",https://vancouver.craigslist.org/van/apa/d/van...,"1022 Nelson Street, Vancouver, BC",1,582.0,2515
1,2022-12-15 12:39,VAN,VAN,Pet Friendly Available Jan. 1st @ Vallejo Cour...,https://vancouver.craigslist.org/van/apa/d/van...,Fairview,1,575.0,2300
2,2022-12-15 12:33,VAN,VAN,BRAND NEW HOUSE 2 BEDROOM BASEMENT FOR RENT IN...,https://vancouver.craigslist.org/van/apa/d/bur...,BURNABY,2,600.0,2300
3,2022-12-15 12:32,VAN,VAN,Large 2bed/2bath Waterfront Unit! Move In Read...,https://vancouver.craigslist.org/van/apa/d/van...,False Creek,2,1400.0,4500
4,2022-12-15 12:24,VAN,VAN,Rarely Available! 2BR/2.5BA Penthouse @ Woodwa...,https://vancouver.craigslist.org/van/apa/d/van...,Gastown,2,1705.0,8000



## Process


* Create and transform data. 
* Maintan data integrity
    * Validility 
    * Accuracy
    * Completeness
    * Consistency
* Test data
### Clean Data
    * Remove duplicate    
    * Find null values then decide how to fill or drop entire row
    * Sorting
    * Filtering
Verify and report on cleaning results

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3050 entries, 0 to 3049
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3050 non-null   object 
 1   City Code      3050 non-null   object 
 2   Area Code      3050 non-null   object 
 3   Post Title     3050 non-null   object 
 4   Post URL       3050 non-null   object 
 5   Neighborhood   3050 non-null   object 
 6   Bedroom        2830 non-null   object 
 7   SQFT           2693 non-null   float64
 8   Price          3050 non-null   int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 214.6+ KB


Convert Post Datetime object to Datetime.

Connect City Code and Area Code then create Post Area Code. (In this DataFrame, it only contain one area of data but I am storeing 13 different area in CSV file. This code will distinguish the data in different regions.)

I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

Sort DataFrame Descending order by Post Datetime.

Remove duplicate. 

Is there any oulier? 

In [7]:
# Convert Datetime then split seperate columns
df['Post Datetime'] = pd.to_datetime(df["Post Datetime"])
df['Post Date'] = df['Post Datetime'].dt.date
df['Post Time'] = df['Post Datetime'].dt.time

df["PostAreaCode"] = df["City Code"] + df["Area Code"]

df = df[["PostAreaCode", 'Post Datetime', 'Post Date', 'Post Time', 'Post Title', 'Post URL', 'Neighborhood', 'Bedroom', 'SQFT', 'Price']]
df.head()

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,VANVAN,2022-12-15 12:41:00,2022-12-15,12:41:00,"2 elevators, 24 hour on-site professional mana...",https://vancouver.craigslist.org/van/apa/d/van...,"1022 Nelson Street, Vancouver, BC",1,582.0,2515
1,VANVAN,2022-12-15 12:39:00,2022-12-15,12:39:00,Pet Friendly Available Jan. 1st @ Vallejo Cour...,https://vancouver.craigslist.org/van/apa/d/van...,Fairview,1,575.0,2300
2,VANVAN,2022-12-15 12:33:00,2022-12-15,12:33:00,BRAND NEW HOUSE 2 BEDROOM BASEMENT FOR RENT IN...,https://vancouver.craigslist.org/van/apa/d/bur...,BURNABY,2,600.0,2300
3,VANVAN,2022-12-15 12:32:00,2022-12-15,12:32:00,Large 2bed/2bath Waterfront Unit! Move In Read...,https://vancouver.craigslist.org/van/apa/d/van...,False Creek,2,1400.0,4500
4,VANVAN,2022-12-15 12:24:00,2022-12-15,12:24:00,Rarely Available! 2BR/2.5BA Penthouse @ Woodwa...,https://vancouver.craigslist.org/van/apa/d/van...,Gastown,2,1705.0,8000


In [8]:
# Convert Bedroom number strings to Integer then NaN value to 0 assume 0 bedroom is studio appartment. 
df["Bedroom"] = pd.to_numeric(df["Bedroom"], errors='ignore').astype('Int64')
df["Bedroom"] = df["Bedroom"].fillna(0)

In [9]:
%%time
# Check if word 'furnished' in the title post. 
search_word = 'furnished'
furnished = []
for i in range(len(df)):
    post_title_list = df['Post Title'][i].lower().split(" ")
    if any(word == search_word for word in post_title_list):
        furnished.append(True)
    else:
        furnished.append(False)
df["IsFurnished"] = furnished

CPU times: total: 0 ns
Wall time: 15.4 ms


In [10]:
# Sort DataFrame by Datetime
df.sort_values(by="Post Datetime", axis=0, ascending=False, inplace=True)

In [11]:
# Remove duplicates
df.drop_duplicates(subset="Post Title", inplace=True)
len(df)

2654

In [12]:
# Find out outlier -Post Price between $1 to $300
df[df["Price"].between(1, 300)]

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,IsFurnished
1472,VANVAN,2022-12-12 13:20:00,2022-12-12,13:20:00,Vancouver Furnished Master Bedroom for Rent,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,400.0,120,True
2272,VANVAN,2022-12-06 21:27:00,2022-12-06,21:27:00,Sublet available Vancouver Dec 19th to Jan 2nd,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,90,False
2609,VANVAN,2022-12-04 15:12:00,2022-12-04,15:12:00,DT One Bedroom One Bath Furnished Dec 21-29 2022,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,650.0,100,True
2852,VANVAN,2022-11-30 20:47:00,2022-11-30,20:47:00,1 Bedroom suite furnished,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver East,1,450.0,150,True
2911,VANVAN,2022-11-29 22:17:00,2022-11-29,22:17:00,Bright Apartment to rent,https://vancouver.craigslist.org/van/apa/d/van...,Kitsilano,1,NaN,100,False
2951,VANVAN,2022-11-29 10:38:00,2022-11-29,10:38:00,1BR Apartment,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,1,628.0,1,False


In [13]:
# One of the low price post (post Price $120)
df.loc[822]["Post URL"]

'https://vancouver.craigslist.org/van/apa/d/vancouver-langara-area-laneway-house/7568119633.html'

This post is daily rental. From this, I assume teh rental price around $100 would be daily rental suite. 

What about $1 or $0 posts?

In [14]:
# $1 ads
one_dollar = df[df["Price"] == 1]
one_dollar["Post URL"]

2951    https://vancouver.craigslist.org/van/apa/d/van...
Name: Post URL, dtype: object

In [15]:
# $0 ads
zero_dollar = df[df["Price"] == 0]
zero_dollar

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,IsFurnished
121,VANVAN,2022-12-15 09:36:00,2022-12-15,09:36:00,Westsea Place - (Westend) - Studio & 1 Bedroom...,https://vancouver.craigslist.org/van/apa/d/van...,1644 NELSON ST.,0,NaN,0,False
574,VANVAN,2022-12-14 10:31:00,2022-12-14,10:31:00,"SCAM $2,045 / 1br - 635ft2 - Suite 1 BR/Quiet ...",https://vancouver.craigslist.org/van/apa/d/van...,kits,1,NaN,0,False
1105,VANVAN,2022-12-13 11:30:00,2022-12-13,11:30:00,DO NOT LIVE HERE - 124 DUNLEVY AVE,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,0,NaN,0,False
2079,VANVAN,2022-12-08 14:02:00,2022-12-08,14:02:00,DO NOT RENT 1310 WEST 10TH AVE,https://vancouver.craigslist.org/van/apa/d/van...,Fairview,1,NaN,0,False
2371,VANVAN,2022-12-06 11:56:00,2022-12-06,11:56:00,Nice Affordable Comfy and Cozy Scams,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,0,NaN,0,False
2393,VANVAN,2022-12-06 01:57:00,2022-12-06,01:57:00,No Cooking is Illegal,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,1,NaN,0,False
2976,VANVAN,2022-11-28 19:50:00,2022-11-28,19:50:00,"SCAM -$1,398 / 1br - Quiet, peaceful 1 BDRM & DEN",https://vancouver.craigslist.org/van/apa/d/van...,"4515 Langara Avenue Vancouver, BC,",0,NaN,0,False
3003,VANVAN,2022-11-28 11:18:00,2022-11-28,11:18:00,Home builder,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,6,NaN,0,False
3026,VANVAN,2022-11-27 20:45:00,2022-11-27,20:45:00,"SCAM - $1,800 - 1br 650 ft furnished short/lon...",https://vancouver.craigslist.org/van/apa/d/nor...,North Vancouver,0,NaN,0,True


In [16]:
# Drop low-end outliers rows
drop_index = []
for idx in one_dollar.index:
    drop_index.append(idx)
for idx in zero_dollar.index:
    drop_index.append(idx)
print(f"Original DataFrame Size:{len(df)}\nNumber of Dropped Rows: {len(drop_index)}")
df.drop(drop_index,axis=0, inplace=True)
print(f"DataFrame size after drop rows: {len(df)}")

Original DataFrame Size:2654
Number of Dropped Rows: 10
DataFrame size after drop rows: 2644


## Analyze - 1

* Find Daily rental suite.
* Dig in high-end rental Market in Vancouver. 

I think it most likely $0 and $1 ads are just not showing the price, so I will drop those rows from DataFrame. 

Tne price around $100 is most likely daily rental so I will store in different variable. 

In [17]:
day_rental = df[df["Price"].between(10, 300)]
day_rental

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,IsFurnished
1472,VANVAN,2022-12-12 13:20:00,2022-12-12,13:20:00,Vancouver Furnished Master Bedroom for Rent,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,400.0,120,True
2272,VANVAN,2022-12-06 21:27:00,2022-12-06,21:27:00,Sublet available Vancouver Dec 19th to Jan 2nd,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,90,False
2609,VANVAN,2022-12-04 15:12:00,2022-12-04,15:12:00,DT One Bedroom One Bath Furnished Dec 21-29 2022,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,650.0,100,True
2852,VANVAN,2022-11-30 20:47:00,2022-11-30,20:47:00,1 Bedroom suite furnished,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver East,1,450.0,150,True
2911,VANVAN,2022-11-29 22:17:00,2022-11-29,22:17:00,Bright Apartment to rent,https://vancouver.craigslist.org/van/apa/d/van...,Kitsilano,1,NaN,100,False


What about high-end outliers?

In [18]:
# 100 higest price, then 2 bedroom suite 
top_100_price = df[['Post Datetime', 'Neighborhood', 'Bedroom', 'SQFT', 'Price', 'IsFurnished']].sort_values(
                by="Price", ascending=False).head(100)
highest_two_bed = top_100_price[top_100_price['Bedroom'] == 2]
highest_two_bed_idx = highest_two_bed.index
highest_two_bed

,Post Datetime,Neighborhood,Bedroom,SQFT,Price,IsFurnished
2444,2022-12-05 20:16:00,Downtown Vancouver,2,800.0,500000,True
355,2022-12-14 16:51:00,Vancouver,2,2694.0,25000,True
2170,2022-12-07 14:56:00,Fairmont Pacific Rim,2,2300.0,18000,False
2763,2022-12-01 21:25:00,"Coal Harbour Front Row, Vancouver",2,2800.0,17900,False
2952,2022-11-29 10:27:00,Downtown Vancouver/Coal Harbour/Waterfront/Sea...,2,2190.0,16900,False
12,2022-12-15 12:11:00,"Vancouver, Point Grey",2,2873.0,15000,True
1476,2022-12-12 13:11:00,Coal Harbour,2,2500.0,14500,False
2369,2022-12-06 12:19:00,Coal Harbour,2,2667.0,14000,False
1502,2022-12-12 11:41:00,Downtown,2,2426.0,13750,False
133,2022-12-15 09:25:00,West Vancouver,2,2641.0,12795,False


In [19]:
# Get URL on 2 Bedroom suite in top 100. 
for idx in highest_two_bed_idx:
    print(f'Index Number - {idx} : {df.loc[idx]["Post URL"]}')


Index Number - 2444 : https://vancouver.craigslist.org/van/apa/d/vancouver-fully-furnished-2bdrm-bath/7557989366.html
Index Number - 355 : https://vancouver.craigslist.org/van/apa/d/vancouver-furnished-two-bedroom-sub/7560020204.html
Index Number - 2170 : https://vancouver.craigslist.org/van/apa/d/vancouver-vancouver-downtown-fairmont/7554964282.html
Index Number - 2763 : https://vancouver.craigslist.org/van/apa/d/rare-harbour-green-one-ft-stunning/7563703745.html
Index Number - 2952 : https://vancouver.craigslist.org/van/apa/d/vancouver-waterfront-luxurious-bedroom/7561685356.html
Index Number - 12 : https://vancouver.craigslist.org/van/apa/d/vancouver-uniqueaccommodationscom/7567894912.html
Index Number - 1476 : https://vancouver.craigslist.org/van/apa/d/vancouver-three-harbour-green-2brden-at/7567600199.html
Index Number - 2369 : https://vancouver.craigslist.org/van/apa/d/vancouver-super-luxury-07-fairmont-bed/7565383014.html
Index Number - 1502 : https://vancouver.craigslist.org/va

#### What is 0.85% RULE?

This is the rough calculation to find out the purchase price to give 5 - 8% cash flow from your rental revenue with today's motgagerate. It's not accurate number but great place to start evaluating the home price for your investment. 

In [20]:
# Average Price on those pent house
avg_rent = np.average(highest_two_bed["Price"][1:].to_numpy())
max_rent = np.max(highest_two_bed["Price"][1:].to_numpy())
# 0.85% RULE
avg_house_p = np.round(avg_rent / (0.85/100), 2)
max_house_p = np.round(max_rent / (0.85/100), 2)
print(f"Price between ${avg_house_p*0.8} - ${max_house_p}")

Price between $1204685.424 - $2941176.47


In [21]:
# Rental price higher than $5000.
more_than_5000 = df[df["Price"] > 5000]
more_than_5000.groupby(by=["Bedroom", "IsFurnished"]).agg({'SQFT': ["min", "max"],
                                                         "Price": ['min', 'max', 'count']})

SQFT           Price              
                        min      max    min     max count
Bedroom IsFurnished                                      
0       True         1357.0   1357.0   8000    8000     1
1       False        2000.0   2000.0   6500    6500     1
        True          900.0   1855.0   5300   10000     6
2       False         675.0   2936.0   5190   18000    90
        True          800.0   2873.0   5200  500000    69
3       False        1169.0   4000.0   5100   20000    79
        True         1050.0   4000.0   5200   24995    30
4       False         900.0   4600.0   5200   25000    36
        True         2000.0   6800.0   5500   18500    14
5       False        1500.0   6426.0   5200   19500    26
        True         2250.0   6500.0   6088   14995     9
6       False        2300.0  10098.0   5600   16500    12
        True         4639.0  29789.0   5600   22880     6
7       False        2900.0   4800.0   6800    9995     5
        True         6000.0   6000.0  19995   19995     1
8       False        4980.0   4980.0   7900    7900     1

In [22]:
# top 10 most expensive rental in Vancouver
top_10 = top_100_price.head(10)
top_10

,Post Datetime,Neighborhood,Bedroom,SQFT,Price,IsFurnished
2444,2022-12-05 20:16:00,Downtown Vancouver,2,800.0,500000,True
355,2022-12-14 16:51:00,Vancouver,2,2694.0,25000,True
1528,2022-12-12 10:56:00,Vancouver,4,3958.0,25000,False
900,2022-12-13 17:48:00,Vancouver,3,2223.0,24995,True
765,2022-12-14 01:08:00,west vancouver,6,10000.0,22880,True
1197,2022-12-13 08:57:00,Vancouver,3,3636.0,20000,False
2526,2022-12-05 09:59:00,Yaletown,3,2223.0,20000,False
1441,2022-12-12 14:50:00,Dowtown,4,3700.0,19998,False
649,2022-12-14 09:26:00,West Vancouver,7,6000.0,19995,True
315,2022-12-14 18:25:00,West Vancouver,3,3600.0,19975,False


In [23]:
# Then get the URL on those top 10 most expensive rental in Vancouver. 
top_10_idx = top_10.index
for idx in top_10_idx:
    print(f'Index Number - {idx}/Number of Bedroom - {df.loc[idx]["Bedroom"]}/SQFT - {df.loc[idx]["SQFT"]}/Location - {df.loc[idx]["Neighborhood"]}\n{df.loc[idx]["Post URL"]}')

Index Number - 2444/Number of Bedroom - 2/SQFT - 800.0/Location - Downtown Vancouver
https://vancouver.craigslist.org/van/apa/d/vancouver-fully-furnished-2bdrm-bath/7557989366.html
Index Number - 355/Number of Bedroom - 2/SQFT - 2694.0/Location - Vancouver
https://vancouver.craigslist.org/van/apa/d/vancouver-furnished-two-bedroom-sub/7560020204.html
Index Number - 1528/Number of Bedroom - 4/SQFT - 3958.0/Location - Vancouver
https://vancouver.craigslist.org/van/apa/d/vancouver-ultra-customized-morden/7567534610.html
Index Number - 900/Number of Bedroom - 3/SQFT - 2223.0/Location - Vancouver
https://vancouver.craigslist.org/van/apa/d/vancouver-luxury-furnished-bed-den-bath/7568077632.html
Index Number - 765/Number of Bedroom - 6/SQFT - 10000.0/Location - west vancouver
https://vancouver.craigslist.org/van/apa/d/west-vancouver-furnished-luxury-mansion/7562366149.html
Index Number - 1197/Number of Bedroom - 3/SQFT - 3636.0/Location - Vancouver
https://vancouver.craigslist.org/van/apa/d/va

From above data, Downtown Vanvouver rental market is different world. I must make decision to define what is the outlier of high-end rental. 

If the number of the high-end prices are large enough, I can make DataFrame for only high-end and search for the high-end market opportunities. 

Clearly, I should not mix with regular market and high-end market because it will provide me biased price average to determin the valuse of the house. 

I am wondering that <a href='https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/Table?TableId=2.1.31.3&GeographyId=2410&GeographyTypeId=3&DisplayAs=Table&GeograghyName=Vancouver'>CMHC Vancouver rental market report</a> concerns about those outliers. 

## Analyze - 2 


## Drop High-end market data then, analyze with regular market. 
* What is the percentage of the Furnished rental suite? 
* Is there any price different? 
* How many bedroom suite has most on ads? And is it any different in area?
* How much is the rental price in general? 


In [32]:
# Drop the rental price over $5000. 
under_5000 = df.drop(more_than_5000.index, axis=0)
print(f'Drop {len(more_than_5000)} rows and data contains {len(under_5000)} rows of data.')

Drop 386 rows and data contains 2258 rows of data.


In [42]:
# then also drop daily rentals
under_5000 = under_5000.drop(day_rental.index, axis=0)
print(f'Drop {len(day_rental)} rows and data contains {len(under_5000)} rows of data.')

Drop 5 rows and data contains 2253 rows of data.


In [43]:
# What is the top 10 highest price on under $5000. Is it no outlier?
under_5000.sort_values(by="Price", ascending=False).head(10)

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,IsFurnished,Weekday
2850,VANVAN,2022-11-30 22:03:00,2022-11-30,22:03:00,Coal Harbour Water View Luxury Condo,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,1255.0,5000,False,3
2967,VANVAN,2022-11-28 23:02:00,2022-11-28,23:02:00,Whole House 2 levels,https://vancouver.craigslist.org/van/apa/d/van...,Arbutus & Dunbar in Vancouver,4,2033.0,5000,False,1
1979,VANVAN,2022-12-09 09:35:00,2022-12-09,09:35:00,Arthur Erickson Waterfall Building 1 Bed + Den...,https://vancouver.craigslist.org/van/apa/d/van...,Armory District,1,1000.0,5000,False,5
1490,VANVAN,2022-12-12 12:24:00,2022-12-12,12:24:00,Yaletown fully furnished 3 bedroom sub-penthou...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,1980.0,5000,True,1
2912,VANVAN,2022-11-29 21:45:00,2022-11-29,21:45:00,(UniqueAccommodations.com) ID:5251 Furnished 2...,https://vancouver.craigslist.org/van/apa/d/van...,"Kitsilano, Vancouver",2,1280.0,5000,True,2
450,VANVAN,2022-12-14 14:00:00,2022-12-14,14:00:00,SPACIOUS WATERFRONT 2BED APARTMENT IN COAL HAR...,https://vancouver.craigslist.org/van/apa/d/van...,"Vancouver, Coal Harbour",2,1300.0,5000,False,3
834,VANVAN,2022-12-13 20:38:00,2022-12-13,20:38:00,"Fully Renovated, Entire House, Available for Rent",https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,3,2000.0,5000,False,2
2919,VANVAN,2022-11-29 18:54:00,2022-11-29,18:54:00,Rare Fully Renovated Luxury Spacious 3 Bedroom...,https://vancouver.craigslist.org/van/apa/d/van...,Yaletown,3,1600.0,5000,False,2
1016,VANVAN,2022-12-13 13:55:00,2022-12-13,13:55:00,A/C Furnished Short term Apartment at Avenue O...,https://vancouver.craigslist.org/van/apa/d/van...,Olympic Village,2,1000.0,5000,True,2
2811,VANVAN,2022-12-01 13:22:00,2022-12-01,13:22:00,Fully furnished Loft that sleeps 6 in Yaletown,https://vancouver.craigslist.org/van/apa/d/van...,yaletown,1,700.0,5000,True,4


In [36]:
# How many rentals are $5000?
under_5000[under_5000["Price"] == 5000] 

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,IsFurnished,Weekday
41,VANVAN,2022-12-15 11:35:00,2022-12-15,11:35:00,5 star luxury Shangri-La hotel condo(furnished...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,1045.0,5000,False,4
450,VANVAN,2022-12-14 14:00:00,2022-12-14,14:00:00,SPACIOUS WATERFRONT 2BED APARTMENT IN COAL HAR...,https://vancouver.craigslist.org/van/apa/d/van...,"Vancouver, Coal Harbour",2,1300.0,5000,False,3
568,VANVAN,2022-12-14 10:42:00,2022-12-14,10:42:00,FRASER AND 41ST VANCOUVER EAST HOUSE 3 BED 2 BATH,https://vancouver.craigslist.org/van/apa/d/van...,"905 E 41st AVE, Vancouver V5W 1P8",3,2000.0,5000,False,3
834,VANVAN,2022-12-13 20:38:00,2022-12-13,20:38:00,"Fully Renovated, Entire House, Available for Rent",https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,3,2000.0,5000,False,2
907,VANVAN,2022-12-13 17:37:00,2022-12-13,17:37:00,(ORCA_REF#1101-1000)2 bdrm 2 bath apt,https://vancouver.craigslist.org/van/apa/d/van...,Downtown Vancouver,2,1350.0,5000,False,2
1016,VANVAN,2022-12-13 13:55:00,2022-12-13,13:55:00,A/C Furnished Short term Apartment at Avenue O...,https://vancouver.craigslist.org/van/apa/d/van...,Olympic Village,2,1000.0,5000,True,2
1086,VANVAN,2022-12-13 12:18:00,2022-12-13,12:18:00,Gorgeous unfurnished 2 bed & den in Coal Harbour,https://vancouver.craigslist.org/van/apa/d/van...,"Coal Harbour, Vancouver, BC",2,1270.0,5000,False,2
1401,VANVAN,2022-12-12 16:27:00,2022-12-12,16:27:00,UBC furnished ocean view 2 bedroom for rent,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,NaN,5000,True,1
1490,VANVAN,2022-12-12 12:24:00,2022-12-12,12:24:00,Yaletown fully furnished 3 bedroom sub-penthou...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,1980.0,5000,True,1
1628,VANVAN,2022-12-11 21:20:00,2022-12-11,21:20:00,NEW BUILDING-Available February 1st -Pet Frien...,https://vancouver.craigslist.org/van/apa/d/van...,Downtown Vancouver,3,745.0,5000,False,7


In [44]:
# What is the cheapest rent?
under_5000.sort_values(by="Price").head(20)

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,IsFurnished,Weekday
1388,VANVAN,2022-12-12 17:06:00,2022-12-12,17:06:00,Short term rental room $400/week,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,1,NaN,400,False,1
1622,VANVAN,2022-12-11 21:39:00,2022-12-11,21:39:00,Double Car Garage for rent !!,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver East Fraserview area,0,400.0,550,False,7
2968,VANVAN,2022-11-28 22:57:00,2022-11-28,22:57:00,Micro-room Joyce Station,https://vancouver.craigslist.org/van/apa/d/van...,Joyce Station Vancouver,1,170.0,550,False,1
1151,VANVAN,2022-12-13 10:16:00,2022-12-13,10:16:00,Unique Basement Suite with High Ceiling,https://vancouver.craigslist.org/van/apa/d/van...,East first ave Vancouver BC,1,150.0,700,False,2
401,VANVAN,2022-12-14 15:26:00,2022-12-14,15:26:00,Nice and cozy room available (H65) (Surrey),https://vancouver.craigslist.org/van/apa/d/sur...,Surrey,1,NaN,700,False,3
3024,VANVAN,2022-11-27 21:08:00,2022-11-27,21:08:00,Im. Looking. Roommate soon as possible please,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,700,False,7
1138,VANVAN,2022-12-13 10:42:00,2022-12-13,10:42:00,Microsuite Available Now,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,150.0,700,False,2
2691,VANVAN,2022-12-03 08:11:00,2022-12-03,08:11:00,1 bedroom available for rent in a shared suite,https://vancouver.craigslist.org/van/apa/d/van...,21st east avenue and Maxwell,1,650.0,750,False,6
971,VANVAN,2022-12-13 15:12:00,2022-12-13,15:12:00,Spot available for one month and one week,https://vancouver.craigslist.org/van/apa/d/van...,Downtown Vancouver,1,NaN,800,False,2
2263,VANVAN,2022-12-06 23:42:00,2022-12-06,23:42:00,Room for rent $830/month! utilities & internet...,https://vancouver.craigslist.org/van/apa/d/van...,vancouver,1,997.0,830,False,2


In [45]:
# Count how many rentals are furnished 
num_furnished_suite = df[df['IsFurnished'] == True]["IsFurnished"].count()
num_unfurnished_suite = len(df) - num_furnished_suite
furnished_suite_percent = np.round((num_furnished_suite / len(df)) * 100, 2)
print(f"Number of furnished suite is {num_furnished_suite}, and it's {furnished_suite_percent}% of the entire list.")

Number of furnished suite is 631, and it's 23.87% of the entire list.


In [46]:
# Find out the number of ads on each bedrooms number.  
num_bed = np.sort(df["Bedroom"].unique())
suite_counts = []
for i in range(len(num_bed)):
    suite_count = len(df[df['Bedroom'] == num_bed[i]])
    suite_counts.append(suite_count)
    #print(f'Number of {num_bed[i]} Bedroom suite: {len(df[df["Bedroom"] == num_bed[i]])}')
#print(f"List of suite counts: {suite_counts}")

In [47]:
# Find out Mean, Max, Min of SQFT and Price of each bedroom and SUM of furnished suite and total suite on each bedroom. 
df_count = df.groupby(by=['Bedroom']).agg({'SQFT': ['mean', 'max', 'min'],
                                                 'Price': ['mean', 'max', 'min'],
                                                  'IsFurnished': ['sum', 'count']})
df_count['percent furnished'] = df_count['IsFurnished']['sum'] / df_count["IsFurnished"]['count']
df_count

SQFT                         Price               IsFurnished  \
                mean      max     min         mean     max   min         sum   
Bedroom                                                                        
0         440.317241   1357.0   103.0  2262.029940    8000   550          51   
1         620.122320   2000.0     1.0  2554.740416   10000    90         254   
2         986.495030   2936.0   500.0  4369.017086  500000  1000         245   
3        1562.544118   4000.0   300.0  5511.532203   24995  1250          50   
4        2619.272727   6800.0   900.0  6591.139535   25000   950          15   
5        3652.675000   6500.0  1500.0  8427.953488   19500  3200           9   
6        6842.166667  29789.0  2300.0  9565.250000   22880   950           6   
7        4028.333333   6000.0  1300.0  9084.285714   19995  1300           1   
8        4980.000000   4980.0  4980.0  7900.000000    7900  7900           0   

              percent furnished  
        count                    
Bedroom                          
0         167          0.305389  
1         913          0.278204  
2        1112          0.220324  
3         295          0.169492  
4          86          0.174419  
5          43          0.209302  
6          20          0.300000  
7           7          0.142857  
8           1          0.000000

In [27]:
# Price per furnished suit on each bedroom
#df_price_furnished = df.drop(['Post Datetime', 'Post Title', 'Post URL', 'Neighborhood'], axis=1)
df_price_furnished = df.groupby(['Bedroom', 'IsFurnished']).agg({'Price': ['mean', 'max', 'min', 'count'],
                                                               "SQFT": ['mean', 'max', 'min']})
df_price_furnished

Price                               SQFT           \
                             mean     max    min count          mean      max   
Bedroom IsFurnished                                                             
0       False         2193.836207    4050    550   116    442.059406   1200.0   
        True          2417.137255    8000   1000    51    436.318182   1357.0   
1       False         2409.371775    6500     90   659    619.803571   2000.0   
        True          2931.897638   10000    100   254    620.888412   1855.0   
2       False         3704.750865   18000   1000   867    967.806701   2936.0   
        True          6719.706122  500000   1175   245   1049.547826   2873.0   
3       False         5141.151020   20000   1250   245   1533.928251   4000.0   
        True          7326.400000   24995   2500    50   1692.775510   4000.0   
4       False         6036.661972   25000    950    71   2550.492063   4600.0   
        True          9215.666667   18500   4800    15   2928.785714   6800.0   
5       False         8168.500000   19500   3200    34   3644.677419   6426.0   
        True          9408.111111   14995   6088     9   3680.222222   6500.0   
6       False         8416.071429   16500    950    14   5079.307692  10098.0   
        True         12246.666667   22880   5600     6  11425.600000  29789.0   
7       False         7265.833333    9995   1300     6   3634.000000   4800.0   
        True         19995.000000   19995  19995     1   6000.000000   6000.0   
8       False         7900.000000    7900   7900     1   4980.000000   4980.0   

                             
                        min  
Bedroom IsFurnished          
0       False         103.0  
        True          180.0  
1       False           1.0  
        True          361.0  
2       False         500.0  
        True          512.0  
3       False         300.0  
        True          781.0  
4       False         900.0  
        True         2000.0  
5       False        1500.0  
        True         2250.0  
6       False        2300.0  
        True         4639.0  
7       False        1300.0  
        True         6000.0  
8       False        4980.0

Dropped extrimely low price add, and it seems no outlier on the price

In [28]:
# Count how many ads per day.1=Mon, 2=Tue, ......, 7=Sun
df["Weekday"] = df["Post Datetime"].dt.isocalendar().day
num_ad_per_day_df = df.groupby('Post Date').agg({'Post Title': 'count',
                                                "Weekday": 'first'})

num_ad_per_day_df

,Post Title,Weekday
Post Date,,
2022-11-27,29,7
2022-11-28,40,1
2022-11-29,46,2
2022-11-30,51,3
2022-12-01,74,4
2022-12-02,57,5
2022-12-03,45,6
2022-12-04,66,7
2022-12-05,135,1


Which day of the week have more ads? 

Is there any day of the week that having more ads than others? 

## Share

#### Data Visualization:
The most important thing is what information do I want to tell. 

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Act